# Importación de librerías

In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix

from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

import funciones_auxiliares as f_aux

seed=123


# Lectura de datasets

In [2]:
pd_loan_train = pd.read_csv("./train_pd_data_preprocessing_missing_outlier.csv").set_index(['SK_ID_CURR'])
pd_loan_test = pd.read_csv("./test_pd_data_preprocessing_missing_outlier.csv").set_index(['SK_ID_CURR'])

C:\Users\Jose Carlos\AppData\Local\Temp\ipykernel_29348\944324200.py:1: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_loan_train = pd.read_csv("./train_pd_data_preprocessing_missing_outlier.csv").set_index(['SK_ID_CURR'])


In [3]:
pd_loan_train.head()

,Unnamed: 0,COMMONAREA_AVG,COMMONAREA_MODE,COMMONAREA_MEDI,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_MEDI,FONDKAPREMONT_MODE,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_AVG,...,FLAG_MOBIL,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
373730,236327,0.0211,0.0191,0.0209,0.0,0.0,0.0,reg oper account,0.0342,0.0336,...,1,-4107,-11719.0,-472,-21057,0.018634,House / apartment,Separated,Higher education,0
127373,23526,0.0211,0.0191,0.0209,0.0,0.0,0.0,No disponible,0.0761,0.0756,...,1,-4808,-2921.0,-239,-15354,0.010032,House / apartment,Married,Higher education,0
265403,142647,0.0211,0.0191,0.0209,0.0,0.0,0.0,No disponible,0.0761,0.0756,...,1,-1565,-6189.0,-1421,-18011,0.035792,House / apartment,Married,Secondary / secondary special,0
441442,294722,0.0211,0.0191,0.0209,0.0,0.0,0.0,No disponible,0.0761,0.0756,...,1,-2442,-2438.0,-856,-11159,0.035792,House / apartment,Married,Secondary / secondary special,0
238432,119381,0.0211,0.0191,0.0209,0.0,0.0,0.0,No disponible,0.0761,0.0756,...,1,-4105,-547.0,-467,-15959,0.018209,House / apartment,Married,Secondary / secondary special,0


In [4]:
pd_loan_train.columns

Index(['Unnamed: 0', 'COMMONAREA_AVG', 'COMMONAREA_MODE', 'COMMONAREA_MEDI',
       'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MODE',
       'NONLIVINGAPARTMENTS_MEDI', 'FONDKAPREMONT_MODE',
       'LIVINGAPARTMENTS_MEDI', 'LIVINGAPARTMENTS_AVG',
       ...
       'FLAG_MOBIL', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED',
       'DAYS_BIRTH', 'REGION_POPULATION_RELATIVE', 'NAME_HOUSING_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE', 'TARGET'],
      dtype='object', length=122)

# Codificación de variables categóricas

In [5]:
cat_vars = pd_loan_train.select_dtypes(include=['object']).columns # Definimos las variables categóricas que tenemos
list_cat_vars = list(cat_vars) # Lo convierto en lista para poder trabajar con ellas posteriormente
unicos = pd_loan_train[cat_vars].nunique() # Observamos cuántos valores únicos tenemos por variable
unicos

FONDKAPREMONT_MODE             5
FLOORSMIN_MODE                26
FLOORSMIN_MEDI                48
ELEVATORS_MODE                27
ELEVATORS_MEDI                47
WALLSMATERIAL_MODE             8
ENTRANCES_MEDI                47
ENTRANCES_MODE                31
HOUSETYPE_MODE                 4
FLOORSMAX_MEDI                49
FLOORSMAX_MODE                26
EMERGENCYSTATE_MODE            3
OCCUPATION_TYPE               19
AMT_REQ_CREDIT_BUREAU_WEEK    10
AMT_REQ_CREDIT_BUREAU_HOUR     6
AMT_REQ_CREDIT_BUREAU_MON     25
AMT_REQ_CREDIT_BUREAU_QRT     11
AMT_REQ_CREDIT_BUREAU_DAY      9
AMT_REQ_CREDIT_BUREAU_YEAR    25
NAME_TYPE_SUITE                8
DEF_30_CNT_SOCIAL_CIRCLE       9
OBS_60_CNT_SOCIAL_CIRCLE      33
OBS_30_CNT_SOCIAL_CIRCLE      33
DEF_60_CNT_SOCIAL_CIRCLE       8
CNT_FAM_MEMBERS               23
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
ORGANIZATION_TYPE             58
NAME_INCOME_TYPE               8
NAME_CONTR

## Separamos X e Y
Donde X es un df sin tener en cuenta la variable objetivo e Y un df solo conteniendo la variable objetivo

- He seleccionado las variables categóricas con **menos de 10 valores únicos para aplicar OneHotEncoding** porque este método crea columnas binarias para cada categoría, lo cual es manejable cuando el número de categorías es bajo. Esto permite representar las categorías de manera explícita sin introducir ningún supuesto sobre la relación con la variable objetivo.

- Por otro lado, he utilizado **Target Encoding para las variables categóricas con más de 10 valores únicos** porque este método reduce la dimensionalidad al codificar las categorías en función de su relación estadística con la variable objetivo. Esto evita el problema de crear demasiadas columnas, que puede llevar a un modelo de mayor complejidad y problemas de memoria.

In [6]:
X_train = pd_loan_train.drop('TARGET',axis=1)
X_test = pd_loan_test.drop('TARGET',axis=1)
y_train = pd_loan_train['TARGET']
y_test = pd_loan_test['TARGET']

In [7]:
list_ohe = []  # Lista vacía para almacenar variables con menos de 10 unicos
for column in list_cat_vars:
    if pd_loan_train[column].nunique() <= 10:
        list_ohe.append(column)

list_ohe

['FONDKAPREMONT_MODE',
 'WALLSMATERIAL_MODE',
 'HOUSETYPE_MODE',
 'EMERGENCYSTATE_MODE',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'NAME_TYPE_SUITE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'NAME_HOUSING_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_EDUCATION_TYPE']

In [8]:
len(list_ohe)

19

In [9]:
ohe = ce.OneHotEncoder(cols=list_ohe, use_cat_names=True)
ohe.fit(X_train, y_train) 

OneHotEncoder(cols=['FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE',
                    'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE',
                    'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_HOUR',
                    'AMT_REQ_CREDIT_BUREAU_DAY', 'NAME_TYPE_SUITE',
                    'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
                    'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
                    'NAME_INCOME_TYPE', 'NAME_CONTRACT_TYPE',
                    'WEEKDAY_APPR_PROCESS_START', 'NAME_HOUSING_TYPE',
                    'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE'],
              use_cat_names=True)

In [10]:
X_train_t = ohe.transform(X_train)
X_test_t = ohe.transform(X_test)

len(list(X_train_t.columns))


213

In [11]:
list_te = []  # Lista vacía para almacenar variables con menos de 10 unicos
for column in list_cat_vars:
    if pd_loan_train[column].nunique() > 10:
        list_te.append(column)

list_te

['FLOORSMIN_MODE',
 'FLOORSMIN_MEDI',
 'ELEVATORS_MODE',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'ENTRANCES_MODE',
 'FLOORSMAX_MEDI',
 'FLOORSMAX_MODE',
 'OCCUPATION_TYPE',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'CNT_FAM_MEMBERS',
 'ORGANIZATION_TYPE']

In [12]:
# Crear una copia de los datos para no modificar los originales
X_train_encoded = X_train_t.copy()
X_test_encoded = X_test_t.copy()

# Inicializar el TargetEncoder
target_enc = ce.TargetEncoder(cols=list_te)

# Ajustar el encoder con los datos de entrenamiento
target_enc.fit(X_train_encoded[list_te], y_train)

# Transformar las columnas seleccionadas
X_train_encoded[list_te] = target_enc.transform(X_train_t[list_te])
X_test_encoded[list_te] = target_enc.transform(X_test_t[list_te])

c:\Users\Jose Carlos\anaconda3\envs\practicaEDA\lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)


In [13]:
X_train_encoded.dtypes.to_dict()

{'Unnamed: 0': dtype('int64'),
 'COMMONAREA_AVG': dtype('float64'),
 'COMMONAREA_MODE': dtype('float64'),
 'COMMONAREA_MEDI': dtype('float64'),
 'NONLIVINGAPARTMENTS_AVG': dtype('float64'),
 'NONLIVINGAPARTMENTS_MODE': dtype('float64'),
 'NONLIVINGAPARTMENTS_MEDI': dtype('float64'),
 'FONDKAPREMONT_MODE_reg oper account': dtype('int64'),
 'FONDKAPREMONT_MODE_No disponible': dtype('int64'),
 'FONDKAPREMONT_MODE_not specified': dtype('int64'),
 'FONDKAPREMONT_MODE_org spec account': dtype('int64'),
 'FONDKAPREMONT_MODE_reg oper spec account': dtype('int64'),
 'LIVINGAPARTMENTS_MEDI': dtype('float64'),
 'LIVINGAPARTMENTS_AVG': dtype('float64'),
 'LIVINGAPARTMENTS_MODE': dtype('float64'),
 'FLOORSMIN_AVG': dtype('float64'),
 'FLOORSMIN_MODE': dtype('float64'),
 'FLOORSMIN_MEDI': dtype('float64'),
 'YEARS_BUILD_AVG': dtype('float64'),
 'YEARS_BUILD_MEDI': dtype('float64'),
 'YEARS_BUILD_MODE': dtype('float64'),
 'OWN_CAR_AGE': dtype('float64'),
 'LANDAREA_MEDI': dtype('float64'),
 'LANDAREA

In [14]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246008 entries, 373730 to 150184
Columns: 213 entries, Unnamed: 0 to NAME_EDUCATION_TYPE_Academic degree
dtypes: float64(62), int64(151)
memory usage: 401.7 MB


Se observa que tenemos todas nuestras variables de tipo numéricas

In [15]:
X_train_encoded.shape

(246008, 213)

Tras realizar los encodings, nuestro dataset de train cuenta con 213 columnas y 246008 registros.

# Escalado de variables

In [16]:
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_encoded)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_encoded), columns=X_train_encoded.columns, index=X_train_encoded.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_encoded), columns=X_test_encoded.columns, index=X_test_encoded.index)

In [17]:
X_train_scaled.describe()

,Unnamed: 0,COMMONAREA_AVG,COMMONAREA_MODE,COMMONAREA_MEDI,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_MEDI,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_No disponible,FONDKAPREMONT_MODE_not specified,...,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Unknown,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree
count,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,...,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,1.137265e-16,-1.223119e-16,-3.389410e-17,1.918403e-16,-2.768427e-17,-3.702067e-17,-2.512813e-17,-8.220077e-17,6.527538e-18,3.526604e-17,...,-1.392987e-16,8.273149e-17,5.053065e-17,7.954354e-17,1.641813e-18,1.026715e-16,2.096177e-17,-2.830525e-17,1.741640e-17,8.679315e-18
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.732203e+00,-6.545080e-01,-6.209154e-01,-6.507466e-01,-1.002067e-01,-9.486810e-02,-9.910172e-02,-5.621899e-01,-1.470614e+00,-1.370730e-01,...,-1.331828e+00,-4.160772e-01,-3.274012e-01,-2.344334e-01,-2.851295e-03,-5.667102e-01,-1.567843e+00,-1.857156e-01,-1.112966e-01,-2.254710e-02
25%,-8.655200e-01,-1.647440e-01,-1.676299e-01,-1.658175e-01,-1.002067e-01,-9.486810e-02,-9.910172e-02,-5.621899e-01,-1.470614e+00,-1.370730e-01,...,-1.331828e+00,-4.160772e-01,-3.274012e-01,-2.344334e-01,-2.851295e-03,-5.667102e-01,-1.567843e+00,-1.857156e-01,-1.112966e-01,-2.254710e-02
50%,-1.224905e-03,-1.647440e-01,-1.676299e-01,-1.658175e-01,-1.002067e-01,-9.486810e-02,-9.910172e-02,-5.621899e-01,6.799879e-01,-1.370730e-01,...,7.508479e-01,-4.160772e-01,-3.274012e-01,-2.344334e-01,-2.851295e-03,-5.667102e-01,6.378190e-01,-1.857156e-01,-1.112966e-01,-2.254710e-02
75%,8.662696e-01,-1.647440e-01,-1.676299e-01,-1.658175e-01,-1.002067e-01,-9.486810e-02,-9.910172e-02,-5.621899e-01,6.799879e-01,-1.370730e-01,...,7.508479e-01,-4.160772e-01,-3.274012e-01,-2.344334e-01,-2.851295e-03,-5.667102e-01,6.378190e-01,-1.857156e-01,-1.112966e-01,-2.254710e-02
max,1.732040e+00,2.255705e+01,2.311131e+01,2.255160e+01,3.696243e+01,3.819668e+01,3.722519e+01,1.778758e+00,6.799879e-01,7.295383e+00,...,7.508479e-01,2.403400e+00,3.054356e+00,4.265604e+00,3.507178e+02,1.764570e+00,6.378190e-01,5.384576e+00,8.985000e+00,4.435160e+01


Los siguientes pasos a seguir serían el **Feature Engineering**, **selección de modelo**, realización de **validación cruzada** y ajuste de **hiperparámetros**.